In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline

# 1. Load Network Data

In [ ]:
data = pd.read_csv("/mnt/idms/temporalNodeRanking/data/filtered_timeline_data/tsv/15o/15o_mentions.csv",sep=" ",names=["time","src","trg"])

In [ ]:
data.head()

# 2. Preprocess Network

## a.) Top authorities

In [ ]:
top_k = 10
top_auth_with_freq = data["trg"].value_counts()[:top_k]
top_auth_with_freq

In [ ]:
top_authorities = list(top_auth_with_freq.index)
top_authorities

## b.) Filter for top authorities

In [ ]:
filtered_data = data[data["trg"].isin(top_authorities)]
print(len(data),len(filtered_data))

In [ ]:
len(filtered_data["trg"].unique())

In [ ]:
targets_for_sources = filtered_data.groupby(by=["src"])["trg"].nunique()

In [ ]:
targets_for_sources.hist()

## c.) Generate group labels for sources